In [189]:
import pandas as pd
import numpy as np
import plotly.express as px
from pandas.api.types import CategoricalDtype

In [190]:
store_data_df = pd.read_csv('../data/raw/dmml1_stores.csv')
test_data_df = pd.read_csv('../data/raw/dmml1_test.csv')
train_data_df = pd.read_csv('../data/raw/dmml1_train.csv')

In [191]:
median_distance = store_data_df['CompetitionDistance'].median()
store_data_df['CompetitionDistance'].fillna(median_distance, inplace=True)

In [192]:
store_data_df

,Store ID,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
1,2,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
2,3,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN
3,4,a,c,1070.0,NaN,NaN,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
4,5,a,c,4590.0,3.0,2000.0,1,40.0,2011.0,"Jan,Apr,Jul,Oct"
...,...,...,...,...,...,...,...,...,...,...
295,296,a,a,690.0,6.0,2007.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
296,297,b,b,720.0,3.0,2002.0,0,NaN,NaN,NaN
297,298,d,c,1340.0,10.0,2006.0,1,5.0,2013.0,"Feb,May,Aug,Nov"
298,299,d,a,260.0,2.0,2012.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"


In [193]:
test_data_df

,Store ID,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,1,1,0,1
1,1,4,2015-07-30,1,1,0,1
2,1,3,2015-07-29,1,1,0,1
3,1,2,2015-07-28,1,1,0,1
4,1,1,2015-07-27,1,1,0,1
...,...,...,...,...,...,...,...
27595,300,2,2015-05-05,1,1,0,0
27596,300,1,2015-05-04,1,1,0,0
27597,300,7,2015-05-03,0,0,0,0
27598,300,6,2015-05-02,1,0,0,0


In [194]:
merged_data = train_data_df.merge(store_data_df, on='Store ID', how='left')

In [195]:
# drop rows Promo2, Promo2SinceWeek, Promo2SinceYear, PromoInterval
merged_data.drop(columns=['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'], inplace=True)

In [196]:
merged_data

,Store ID,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2
0,1,4,2015-04-30,6884,716,1,1,0,0,a,a,570.0,1
1,1,3,2015-04-29,6764,756,1,1,0,0,a,a,570.0,1
2,1,2,2015-04-28,6861,678,1,1,0,0,a,a,570.0,1
3,1,1,2015-04-27,6523,647,1,1,0,0,a,a,570.0,1
4,1,7,2015-04-26,0,0,0,0,0,0,a,a,570.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
246898,300,6,2013-01-05,5194,569,1,0,0,0,a,c,9260.0,0
246899,300,5,2013-01-04,5524,646,1,0,0,1,a,c,9260.0,0
246900,300,4,2013-01-03,5563,718,1,0,0,1,a,c,9260.0,0
246901,300,3,2013-01-02,6218,753,1,0,0,1,a,c,9260.0,0


# Empfehlung für den Umgang mit großen Werten

Capping: Eine Möglichkeit wäre, die Werte am 95. oder 99. Perzentil zu begrenzen. Dies würde bedeuten, dass Werte, die diese Schwellen überschreiten, auf den Wert des entsprechenden Perzentils reduziert werden.

Logarithmische Transformation: Eine logarithmische Transformation könnte die rechtsschiefe Verteilung normalisieren und den Einfluss von extremen Werten reduzieren.

Binarisierung: Die Umwandlung in eine binäre Variable (z.B. "nah" vs. "fern") könnte ebenfalls in Betracht gezogen werden, insbesondere wenn Sie vermuten, dass die Entfernung des Wettbewerbers jenseits eines bestimmten Schwellenwerts keinen signifikanten Einfluss mehr hat.

In [197]:
# Anpassung der Datumsmerkmale
merged_data['Date'] = pd.to_datetime(merged_data['Date'])
merged_data['Year'] = merged_data['Date'].dt.year
merged_data['Month'] = merged_data['Date'].dt.month
merged_data['Day'] = merged_data['Date'].dt.day
merged_data['WeekOfYear'] = merged_data['Date'].dt.isocalendar().week
merged_data['Weekend'] = np.where(merged_data['DayOfWeek'].isin([6, 7]), 1, 0)  # Samstag = 6, Sonntag = 7

cat_type = CategoricalDtype(categories=['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Samstag', 'Sonntag'], ordered=True)
merged_data['Weekday'] = merged_data['Date'].dt.day_name(locale='de_DE').astype(cat_type)

merged_data['Quarter'] = merged_data['Date'].dt.quarter
merged_data['DayOfYear'] = merged_data['Date'].dt.dayofyear
merged_data['DayOfMonth'] = merged_data['Date'].dt.day

merged_data['Season'] = merged_data['Month'].apply(lambda month: (month%12 // 3 + 1))
merged_data['Season'].replace(to_replace=[1,2,3,4], value=['Winter', 'Frühling','Sommer','Herbst'], inplace=True)

# Entfernen der ursprünglichen Date-Spalte
merged_data.drop('Date', axis=1, inplace=True)

In [202]:
from sklearn.preprocessing import OneHotEncoder

In [203]:
# Auswahl der kategorischen Spalten für One-Hot-Encoding
categorical_columns = ['StateHoliday', 'StoreType', 'Assortment', 'Season', 'Weekday']

# Anwendung von One-Hot-Encoding auf kategorische Spalten
encoder = OneHotEncoder(sparse=False, drop='first')
encoded_columns = encoder.fit_transform(merged_data[categorical_columns])

# Umwandlung des encoded Arrays in einen DataFrame und Anpassung der Spaltennamen
encoded_columns_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(categorical_columns))

# Zusammenführen des encoded DataFrame mit dem ursprünglichen Datensatz
merged_data.reset_index(drop=True, inplace=True)
merged_data_encoded = pd.concat([merged_data, encoded_columns_df], axis=1)

# Entfernen der ursprünglichen kategorischen Spalten
merged_data_encoded.drop(categorical_columns, axis=1, inplace=True)



/Users/wladi/opt/anaconda3/envs/bv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Store ID,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday,CompetitionDistance,Promo2,Year,...,Assortment_c,Season_Herbst,Season_Sommer,Season_Winter,Weekday_Donnerstag,Weekday_Freitag,Weekday_Mittwoch,Weekday_Montag,Weekday_Samstag,Weekday_Sonntag
0,1,4,6884,716,1,1,0,570.0,1,2015,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,3,6764,756,1,1,0,570.0,1,2015,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,2,6861,678,1,1,0,570.0,1,2015,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,6523,647,1,1,0,570.0,1,2015,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1,7,0,0,0,0,0,570.0,1,2015,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [206]:
from sklearn.preprocessing import StandardScaler

# Auswahl der numerischen Spalten für die Standardisierung (ausschließen der binären Spalten)
numerical_columns = [col for col in merged_data_encoded.columns 
                     if col not in ['Open', 'Promo', 'SchoolHoliday', 'Promo2', 'Weekend'] 
                     and merged_data_encoded[col].nunique() > 2]

# Anwendung der Standardisierung nur auf numerische Spalten
scaler = StandardScaler()
scaled_numerical = scaler.fit_transform(merged_data_encoded[numerical_columns])

# Ersetzen der ursprünglichen numerischen Spalten im DataFrame durch die skalierten Werte
scaled_numerical_df = pd.DataFrame(scaled_numerical, columns=numerical_columns)
for col in numerical_columns:
    merged_data_encoded[col] = scaled_numerical_df[col]

# Anzeigen der ersten paar Zeilen des skalierten Datensatzes
merged_data_encoded.head()


,Store ID,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday,CompetitionDistance,Promo2,Year,...,Assortment_c,Season_Herbst,Season_Sommer,Season_Winter,Weekday_Donnerstag,Weekday_Freitag,Weekday_Mittwoch,Weekday_Montag,Weekday_Samstag,Weekday_Sonntag
0,-1.726101,0.001553,0.259423,0.161865,1,1,0,-0.621747,1,1.833954,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,-1.726101,-0.498947,0.229475,0.246522,1,1,0,-0.621747,1,1.833954,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,-1.726101,-0.999446,0.253683,0.081441,1,1,0,-0.621747,1,1.833954,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-1.726101,-1.499946,0.169330,0.015832,1,1,0,-0.621747,1,1.833954,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,-1.726101,1.503051,-1.458568,-1.353497,0,0,0,-0.621747,1,1.833954,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [209]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Angenommen, dass 'y' Ihre Zielvariable ist
y = merged_data_encoded['Sales']
X = merged_data_encoded.drop(['Sales', 'Customers'], axis=1)

# Aufteilen der Daten in Trainings- und Testsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Erstellen und Trainieren des linearen Regressionsmodells
model = LinearRegression()
model.fit(X_train, y_train)

# Vorhersagen auf dem Testset
y_pred = model.predict(X_test)

# Berechnen des RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
# Berechnen des R-Quadrat-Werts
r_squared = model.score(X_test, y_test)

print(f'R-squared: {r_squared}')
print(f'Root Mean Squared Error: {rmse}')

R-squared: 0.5527850703457732
Root Mean Squared Error: 0.6689602477561157


Die beiden Werte, die Sie erhalten haben, sind Metriken zur Bewertung Ihres Modells:

1. **R-Quadrat (R-squared)**: Dieser Wert reicht von 0 bis 1 und gibt an, welcher Anteil der Varianz in der Zielvariable durch das Modell erklärt wird. Ein Wert von 1 bedeutet, dass das Modell alle Variationen in den Daten erklärt, während ein Wert von 0 bedeutet, dass das Modell keine der Variationen erklärt. In Ihrem Fall erklärt das Modell etwa 55.3% der Variation in den Verkaufsdaten.

2. **Root Mean Squared Error (RMSE)**: Dies ist der Quadratwurzel des Durchschnitts der quadratischen Unterschiede zwischen den vorhergesagten und den tatsächlichen Werten. Es misst im Grunde genommen den durchschnittlichen Fehler, den das Modell macht. Ein niedrigerer Wert bedeutet ein besseres Modell. In Ihrem Fall beträgt der durchschnittliche Fehler des Modells etwa 0.669 (angenommen, dass Ihre Zielvariable normalisiert wurde, da der RMSE nahe bei 0 liegt).
